<a href="https://colab.research.google.com/github/Azad11014/MNIST-/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

#Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


#Preprocessing

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_sample = 0.1 * mnist_info.splits['train'].num_examples
num_validation_sample = tf.cast(num_validation_sample, tf.int64)

num_test_sample = mnist_info.splits['test'].num_examples

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

#Shuffling and Batching
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_sample)
train_data = shuffled_train_and_validation_data.skip(num_validation_sample)

#Mini batch GD
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_sample)
test_data = test_data.batch(num_test_sample)
validation_input, validation_target = next(iter(validation_data))

#Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

#Optimizer and loss function

In [5]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

#Training

In [6]:
NUM_EPOCH = 10
model.fit(train_data, epochs = NUM_EPOCH, validation_data = (validation_input, validation_target))

Epoch 1/10
540/540 [==============================] - 9s 11ms/step - loss: 0.4073 - accuracy: 0.8841 - val_loss: 0.2152 - val_accuracy: 0.9360
Epoch 2/10
540/540 [==============================] - 5s 8ms/step - loss: 0.1820 - accuracy: 0.9473 - val_loss: 0.1539 - val_accuracy: 0.9537
Epoch 3/10
540/540 [==============================] - 4s 6ms/step - loss: 0.1401 - accuracy: 0.9591 - val_loss: 0.1193 - val_accuracy: 0.9660
Epoch 4/10
540/540 [==============================] - 4s 6ms/step - loss: 0.1161 - accuracy: 0.9655 - val_loss: 0.1037 - val_accuracy: 0.9723
Epoch 5/10
540/540 [==============================] - 5s 9ms/step - loss: 0.0952 - accuracy: 0.9720 - val_loss: 0.0873 - val_accuracy: 0.9743
Epoch 6/10
540/540 [==============================] - 4s 6ms/step - loss: 0.0823 - accuracy: 0.9749 - val_loss: 0.0869 - val_accuracy: 0.9753
Epoch 7/10
540/540 [==============================] - 4s 6ms/step - loss: 0.0718 - accuracy: 0.9786 - val_loss: 0.0733 - val_accuracy: 0.9790
Epoch

#Testing

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0884 - accuracy: 0.9730


In [8]:
print(f'Test Loss: {test_loss:.2f}. Test Accuracy: {test_accuracy * 100:.2f}%')

Test Loss: 0.09. Test Accuracy: 97.30%


#Saving the model

In [22]:
model.save('MNIST.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Testing with actual data

In [25]:
import numpy as np
from PIL import Image
import tensorflow as tf

def preprocess_image(image_path):
    # Load the image
    img = Image.open(image_path)
    img.show()
    # Convert to grayscale
    img = img.convert('L')

    # Resize to 28x28 pixels (the size of MNIST images)
    img = img.resize((28, 28))

    # Convert the image to a numpy array
    img_array = np.array(img)

    # Normalize the pixel values to be between 0 and 1
    img_array = img_array / 255.0

    # Invert the image colors (MNIST has white digits on black background)
    img_array = 1 - img_array

    # Reshape the image to match the input shape expected by the model
    img_array = img_array.reshape(1, 28, 28, 1)


    return img_array

image_path = '/two.jpeg'  # Replace with your image path
processed_image = preprocess_image(image_path)
model = tf.keras.models.load_model('MNIST.h5')  # Replace with your model path

prediction = model.predict(processed_image)
predicted_digit = np.argmax(prediction)

print(f'The predicted digit is: {predicted_digit}')

1/1 [==============================] - 0s 60ms/step
The predicted digit is: 2


#Saving the model in drive

In [24]:
from google.colab import drive
import tensorflow as tf

# Mount Google Drive
drive.mount('/content/drive')

# Assume your model is already trained and stored in a variable called 'model'

# Save the model to Google Drive
model.save('/content/drive/My Drive/MNIST_AZAD.h5')

# Specify the path
saved_model_path = '/content/drive/My Drive/MNIST_AZAD.h5'

print("Model saved at:", saved_model_path)


Mounted at /content/drive
Model saved at: /content/drive/My Drive/MNIST_AZAD.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
